# Deploy and inference Code Embeddings 0.5b/1.5b with Azure Virtual machine

This notebook demonstrates how to deploy a [Jina Code Embeddings](https://jina.ai/news/jina-code-embeddings-sota-code-retrieval-at-0-5b-and-1-5b) model-powered [Azure Virtual machine (0.5b)](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/jinaai.jina-code-embeddings-500m?tab=Overview) / [Azure Virtual machine (1.5b)](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/jinaai.jina-code-embeddings-1500m?tab=Overview) and perform inference with the application within the virtual machine.

## Deploy the virtual machine

To deploy the virtual machine, start by consulting the [official deployment guide](https://learn.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-portal). This document provides comprehensive steps for the deployment process.

It's worth mentioning that in the Basics tab of the VM setup, you will need to provide several details about the VM. 

You can customize the VM size used, and for certain sizes, you might need to adjust the allowed quota to ensure access. It is recommended to use the [Standard_NC4as_T4_v3](https://learn.microsoft.com/en-us/azure/virtual-machines/nct4-v3-series) VM. This VM features up to 1 NVIDIA T4 GPU with 16 GB of memory.

For the other tabs, you can leave most settings as default or adjust them to fit your needs.

<img src="images/deploy_embedding_v4.png" width="50%" height="50%">

Once the deployment of the VM is complete, proceed to the resource group created for your deployment to verify the resources that have been established. 

Within the resource group, you'll find the public IP through which you can access your application within the VM.

Please note that the application within the VM will be unavailable immediately after deployment due to necessary model loading process **We recommend waiting at least 2 minutes before using the application.**

# Perform inference with the application

The Python example below demonstrates how to perform real-time inference using the public IP address of the deployed virtual machine.

In [ ]:
import json

import requests


def invoke_endpoint():
    url = "http://<Insert here your public IP address>/encode"  # For example: http://20.84.48.180/encode
    headers = {"Content-Type": "application/json"}
    json_data =   {
        "data": [
            "Calculates the square of a number. Parameters: number (int or float) - The number to square. Returns: int or float - The square of the number.",
            "This function calculates the square of a number you give it.",
            "def square(number): return number ** 2",
            "print(square(5))",
            "Output: 25",
            "Each text can be up to 8192 tokens long"
            ],
        "parameters": {
            # Select the downstream task for which the embeddings will be used.
            # The model will return the optimized embeddings for that task.
            # Options: nl2code.query, nl2code.passage, code2code.query, code2code.passage,
            # code2nl.query, code2nl.passage, code2completion.query, code2completion.passage,
            # qa.query, qa.passage
            "task": "nl2code.passage"
        }
  }

    response = requests.post(url, headers=headers, data=json.dumps(json_data))
    print(response.json())


invoke_endpoint()